In [110]:
import pandas as pd

In [111]:
df=pd.read_csv(r"V:\Downloads\Demo_sales_data.csv")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2021,11/11/2021,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.91
1,2,CA-2016-152156,08/11/2021,11/11/2021,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.58
2,3,CA-2016-138688,12/06/2021,16/06/2021,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.87
3,14,CA-2016-161389,05/12/2021,10/12/2021,Standard Class,IM-15070,Irene Maddox,Consumer,United States,Seattle,...,98103,West,OFF-BI-10003656,Office Supplies,Binders,Fellowes PB200 Plastic Comb Binding Machine,407.98,3,0.2,132.59
4,22,CA-2016-137330,09/12/2021,13/12/2021,Standard Class,KB-16585,Ken Black,Corporate,United States,Fremont,...,68025,Central,OFF-AR-10000246,Office Supplies,Art,Newell 318,19.46,7,0.0,5.06


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2587 entries, 0 to 2586
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         2587 non-null   int64  
 1   Order ID       2587 non-null   object 
 2   Order Date     2587 non-null   object 
 3   Ship Date      2587 non-null   object 
 4   Ship Mode      2587 non-null   object 
 5   Customer ID    2587 non-null   object 
 6   Customer Name  2587 non-null   object 
 7   Segment        2587 non-null   object 
 8   Country        2587 non-null   object 
 9   City           2587 non-null   object 
 10  State          2587 non-null   object 
 11  Postal Code    2587 non-null   int64  
 12  Region         2587 non-null   object 
 13  Product ID     2587 non-null   object 
 14  Category       2587 non-null   object 
 15  Sub-Category   2587 non-null   object 
 16  Product Name   2587 non-null   object 
 17  Sales          2587 non-null   float64
 18  Quantity

In [113]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
df.drop(columns=['Row ID','Order ID','Ship Date','Ship Mode','City','State','Region','Profit','Customer Name','Segment','Country','Postal Code','Product ID','Category','Sub-Category','Product Name','Discount'],inplace=True)

In [114]:
reference_date = pd.to_datetime('2021-12-31')

In [115]:
df.head()

,Order Date,Customer ID,Sales,Quantity
0,2021-11-08,CG-12520,261.96,2
1,2021-11-08,CG-12520,731.94,3
2,2021-06-12,DV-13045,14.62,2
3,2021-12-05,IM-15070,407.98,3
4,2021-12-09,KB-16585,19.46,7


In [116]:
aggregated_df = df.groupby('Customer ID').agg({
    'Sales': 'sum',
    'Order Date': lambda x: (reference_date - x.max()).days,
    'Customer ID': 'size'
}).rename(columns={'Order Date': 'recency', 'Customer ID': 'frequency', 'Sales': 'monetary_value'})

In [117]:
aggregated_df.head()

,monetary_value,recency,frequency
Customer ID,,,
AA-10315,4406.07,303,4
AA-10375,530.29,47,4
AA-10480,1747.50,127,10
AA-10645,1668.19,118,8
AB-10015,563.94,51,2


In [118]:
quartiles = aggregated_df.quantile(q=[0.25,0.50,0.75])
print(quartiles, type(quartiles))
quartiles=quartiles.to_dict()
quartiles
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

      monetary_value  recency  frequency
0.25        107.2775     32.0        2.0
0.50        495.7300     72.0        3.0
0.75       1210.8250    148.5        6.0 <class 'pandas.core.frame.DataFrame'>


In [119]:
rfmSeg = aggregated_df
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfmSeg['M_Quartile'] = rfmSeg['monetary_value'].apply(FMClass, args=('monetary_value',quartiles,))

In [120]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str)+rfmSeg.F_Quartile.map(str)+rfmSeg.M_Quartile.map(str)

In [121]:
rfmSeg.head()

,monetary_value,recency,frequency,R_Quartile,F_Quartile,M_Quartile,RFMClass
Customer ID,,,,,,,
AA-10315,4406.07,303,4,4,2,1,421
AA-10375,530.29,47,4,2,2,2,222
AA-10480,1747.50,127,10,3,1,1,311
AA-10645,1668.19,118,8,3,1,1,311
AB-10015,563.94,51,2,2,4,2,242


In [122]:
rfmSeg['Total Score'] = rfmSeg['R_Quartile'] + rfmSeg['F_Quartile'] +rfmSeg['M_Quartile']

In [123]:
rfmSeg.sort_values(by='Total Score',ascending=False).head(10)

,monetary_value,recency,frequency,R_Quartile,F_Quartile,M_Quartile,RFMClass,Total Score
Customer ID,,,,,,,,
PG-18895,39.12,253,2,4,4,4,444,12
TH-21100,27.86,236,1,4,4,4,444,12
RA-19285,3.98,149,1,4,4,4,444,12
BE-11335,102.92,159,2,4,4,4,444,12
BD-11725,46.72,309,1,4,4,4,444,12
RE-19450,29.61,170,1,4,4,4,444,12
LB-16735,21.48,159,1,4,4,4,444,12
BB-10990,61.70,274,2,4,4,4,444,12
PS-18970,8.00,236,1,4,4,4,444,12


In [129]:
rfmSeg.groupby('Total Score').agg({'monetary_value':'mean','recency':'mean','frequency':'mean','RFMClass':['min','max']})

monetary_value     recency  frequency RFMClass     
                      mean        mean       mean      min  max
Total Score                                                    
3              2690.095294   15.558824  11.117647      111  111
4              1978.730345   28.672414   7.517241      112  211
5              2597.510962   50.346154   6.326923      113  311
6              1209.078293   61.951220   5.134146      123  411
7              1025.152034   86.932203   4.186441      124  421
8               557.317093  103.325581   3.523256      134  431
9               421.241867  105.693333   2.360000      144  441
10              254.817595  134.164557   1.797468      244  442
11              110.245614  166.421053   1.526316      344  443
12               41.791071  236.267857   1.196429      444  444